# Wav2Vec2-Large-Ru-Golos

##### https://huggingface.co/bond005/wav2vec2-large-ru-golos

Модель Wav2Vec2 основана на `facebook/wav2vec2-large-xlsr-53`, доработана на русском языке с помощью `Sberdevices Gloss` с дополнениями для звука, такими как сдвиг высоты тона, ускорение / замедление звука, реверберация и т.д.

При использовании этой модели убедитесь, что частота дискретизации речевого ввода составляет 16 кГц.

## Подключение библиотеки

In [1]:
import os
import warnings

import torch
from datasets import load_dataset, load_from_disk
from datasets.features import Audio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import pandas as pd
import torchaudio

# Для теста WER - Word Error Rate
# CER - Character Error Rate
# MER - Match Error Rate
# WIL - Word Information Lost
from jiwer import wer, cer, mer, wil

## Использование CUDA

In [2]:
# Check if CUDA is available
if torch.cuda.is_available():
    # Get the device name
    device = torch.cuda.current_device()
    print(f"Code is connected to CUDA. Using GPU: {torch.cuda.get_device_name(device)}")
else:
    print("CUDA is not available. Running on CPU.")

Code is connected to CUDA. Using GPU: NVIDIA GeForce GTX 1650 Ti


In [3]:
# Все равно у меня не хватит памяти для выполнения(
device = 'cpu'

Не загружаем модель через CUDA, памяти не хватило

`RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 3.63 GiB total capacity; 2.56 GiB already allocated; 7.12 MiB free; 2.58 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF`

In [4]:
# Создадим датафрейм, который будет собирать данные и сохранять
info = pd.DataFrame()

## Загрузка модели

In [5]:
LANG_ID = "ru"
MODEL_ID = 'bond005/wav2vec2-large-ru-golos'
PATH_MODEL = '/home/redalexdad/recognition_speech/wav2vec2-large-ru-golos'
# Кол-во текстов для предсказания
# ВНИМАНИЕ, НЕ СТАВЬ БОЛЬШЕ 10, ОС ЗАВИСНЕТ И ЯДРО УПАДЕТ!
SAMPLES = 3

In [6]:
# Проверка наличия модели в локальном пути
if not os.path.exists(PATH_MODEL):
    processor = Wav2Vec2Processor.from_pretrained(PATH_MODEL)
    model = Wav2Vec2ForCTC.from_pretrained(PATH_MODEL).to(device)
    print('Успешно модель загружена')
else:
    # Загрузка процессора из сети
    processor = Wav2Vec2Processor.from_pretrained("bond005/wav2vec2-large-ru-golos")
    processor.save_pretrained(PATH_MODEL)
    
    # Загрузка модели из сети
    model = Wav2Vec2ForCTC.from_pretrained("bond005/wav2vec2-large-ru-golos").to(device)
    model.save_pretrained(PATH_MODEL)
    
    print(f'Успешно процессор и модель скачаны и сохранены в пути {PATH_MODEL}')

Успешно процессор и модель скачаны и сохранены в пути /home/redalexdad/recognition_speech/wav2vec2-large-ru-golos


## Загрузка датасета и небольшая проверка `sberdevices_golos_10h_crowd`

In [7]:
# Загрузка тестовую часть набора данных Golos и чтение первого звукового файла
ds = load_dataset("bond005/sberdevices_golos_10h_crowd", split="test")

In [8]:
# Токенизация
# Batch size 1
processed = processor(ds[0]["audio"]["array"], return_tensors="pt", padding="longest").to(device)

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
2024-03-11 01:12:41.615715: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 01:12:41.615770: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 01:12:41.692041: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-11 01:12:41.853494: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the follow

In [9]:
# Освобождение памяти на GPU
torch.cuda.empty_cache()

In [10]:
# Извлекаем логиты
logits = model(processed.input_values, attention_mask=processed.attention_mask).logits

In [11]:
# Транскрибация
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)[0]
print(transcription)

шестьдесят тысяч тенге сколько будет стоить


## Загрузка датасета `common_voice_11_0`

In [12]:
%%time
test_dataset_cv_11 = load_dataset("mozilla-foundation/common_voice_11_0", LANG_ID, split=f"test[:{SAMPLES}]", trust_remote_code=True)

CPU times: user 345 ms, sys: 3.76 ms, total: 349 ms
Wall time: 7.53 s


In [13]:
if test_dataset_cv_11.features['audio'].sampling_rate != 16_000:
    test_dataset_cv_11 = test_dataset_cv_11.cast_column(
        'audio',
        Audio(sampling_rate=16_000)
    )

### Предсказание

In [14]:
# Освобождаем памяти
torch.cuda.empty_cache()

In [15]:
%%time
audio_data = [test_dataset_cv_11[i]['audio']['array'] for i in range(SAMPLES)]

processed = processor(audio_data, sampling_rate=16_000, return_tensors="pt", padding='longest').to(device)

CPU times: user 15.8 ms, sys: 6.25 ms, total: 22.1 ms
Wall time: 22.9 ms


In [16]:
# Извлекаем логиты напрямую с использованием model.forward
logits = model(processed.input_values, attention_mask=processed.attention_mask).logits

In [17]:
# Оригинальные тексты из датасета
original_texts = [test_dataset_cv_11[i]['sentence'] for i in range(SAMPLES)]

In [18]:
# Транскрибация
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

### Вывод результатов

In [19]:
# Вывод оригинальных и предсказанных текстов
print('-'*100)
for i, (original_text, predicted_text) in enumerate(zip(original_texts, transcription)):
    print(f"Sample {i + 1} \n- Канонический текст: {original_text}")
    print(f"- Предсказанный текст: {predicted_text}")
    print("-" * 100)

----------------------------------------------------------------------------------------------------
Sample 1 
- Канонический текст: К сожалению, эти предложения не нашли отражения в тексте.
- Предсказанный текст: к сожалению эти предложений не носли отражения в тексте
----------------------------------------------------------------------------------------------------
Sample 2 
- Канонический текст: Если не будет возражений, я буду считать, что Ассамблея согласна с этим предложением.
- Предсказанный текст: если не будет возражений я буду считать что ассамблея согласена с этим предложением
----------------------------------------------------------------------------------------------------
Sample 3 
- Канонический текст: Новошахтинск — милый город
- Предсказанный текст: новошахкинск милый город
----------------------------------------------------------------------------------------------------


### Тест WER, CER, MER, WIL

In [20]:
# Приведем все к нижнему регистру
original_texts = [ref.lower() for ref in original_texts]
transcription = [pred.lower() for pred in transcription]

In [21]:
original_texts

['к сожалению, эти предложения не нашли отражения в тексте.',
 'если не будет возражений, я буду считать, что ассамблея согласна с этим предложением.',
 'новошахтинск — милый город']

In [21]:
# Рассчитаем WER
wer_score = wer(original_texts, transcription)

In [22]:
print(f"Word Error Rate (WER): {wer_score * 100:.2f}%")

Word Error Rate (WER): 38.46%


In [23]:
# и CER
cer_score = cer(original_texts, transcription)

In [24]:
print(f"Character Error Rate (CER): {cer_score * 100:.2f}%")

Character Error Rate (CER): 7.14%


In [25]:
# MER
mer_score = mer(original_texts, transcription)

In [26]:
print(f"Match Error Rate (MER): {mer_score * 100:.2f}%")

Match Error Rate (MER): 38.46%


In [27]:
# WIL
wil_score = wil(original_texts, transcription)

In [28]:
print(f"Word Information Lost (WIL): {wil_score * 100:.2f}%")

Word Information Lost (WIL): 60.62%


In [29]:
# Создание датафрейма
info = info.append({
    'MODEL': MODEL_ID,
    'DATASET': test_dataset_cv_11.info.dataset_name,
    'ORIGINAL TEXT': original_texts,
    'PREDICTION TEXT': transcription,
    'WER': wer_score,
    'CER': cer_score,
    'MER': mer_score,
    'WIL': wil_score,
    'SAMPLES': SAMPLES
}, ignore_index=True)
display(info)

/tmp/ipykernel_22161/3376670737.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  info = info.append({


,MODEL,DATASET,ORIGINAL TEXT,PREDICTION TEXT,WER,CER,MER,WIL,SAMPLES
0,bond005/wav2vec2-large-ru-golos,common_voice_11_0,"[к сожалению, эти предложения не нашли отражен...",[к сожалению эти предложений не носли отражени...,0.384615,0.071429,0.384615,0.606154,3


## Загрузка датасета `common_voice_12_0`

In [30]:
%%time
test_dataset_cv_12 = load_dataset("mozilla-foundation/common_voice_12_0", LANG_ID, split=f"test[:{SAMPLES}]", trust_remote_code=True)

CPU times: user 296 ms, sys: 14.3 ms, total: 310 ms
Wall time: 5.82 s


In [31]:
if test_dataset_cv_12.features['audio'].sampling_rate != 16_000:
    test_dataset_cv_12 = test_dataset_cv_12.cast_column(
        'audio',
        Audio(sampling_rate=16_000)
    )

### Предсказание

In [32]:
# Освобождаем памяти 50
torch.cuda.empty_cache()

In [33]:
%%time
audio_data = [test_dataset_cv_12[i]['audio']['array'] for i in range(SAMPLES)]

processed = processor(audio_data, sampling_rate=16_000, return_tensors="pt", padding='longest').to(device)

CPU times: user 31.4 ms, sys: 0 ns, total: 31.4 ms
Wall time: 95.6 ms


In [34]:
# Извлекаем логиты напрямую с использованием model.forward
logits = model(processed.input_values, attention_mask=processed.attention_mask).logits

In [35]:
# Оригинальные тексты из датасета
original_texts = [test_dataset_cv_12[i]['sentence'] for i in range(SAMPLES)]

In [36]:
# Транскрибация
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

### Вывод результатов

In [37]:
# Вывод оригинальных и предсказанных текстов
print('-'*100)
for i, (original_text, predicted_text) in enumerate(zip(original_texts, transcription)):
    print(f"Sample {i + 1} \n- Канонический текст: {original_text}")
    print(f"- Предсказанный текст: {predicted_text}")
    print("-" * 100)

----------------------------------------------------------------------------------------------------
Sample 1 
- Канонический текст: К сожалению, эти предложения не нашли отражения в тексте.
- Предсказанный текст: к сожалению эти предложений не носли отражения в тексте
----------------------------------------------------------------------------------------------------
Sample 2 
- Канонический текст: Толпа озвереет, будет тереться, ощетинит ножки стоглавая вошь.
- Предсказанный текст: толпа звереет будет тереться ощути нетножки стоглавая вожь
----------------------------------------------------------------------------------------------------
Sample 3 
- Канонический текст: А жизнь ее была не весела.
- Предсказанный текст: а жизнь ее была невесела
----------------------------------------------------------------------------------------------------


### Тест WER, CER

In [38]:
# Приведем все к нижнему регистру
original_texts = [ref.lower() for ref in original_texts]
transcription = [pred.lower() for pred in transcription]

In [39]:
# Рассчитаем WER
wer_score = wer(original_texts, transcription)

In [40]:
print(f"Word Error Rate (WER): {wer_score * 100:.2f}%")

Word Error Rate (WER): 47.83%


In [41]:
# и CER
cer_score = cer(original_texts, transcription)

In [42]:
print(f"Character Error Rate (CER): {cer_score * 100:.2f}%")

Character Error Rate (CER): 11.03%


In [43]:
# MER
mer_score = mer(original_texts, transcription)

In [44]:
print(f"Match Error Rate (MER): {mer_score * 100:.2f}%")

Match Error Rate (MER): 47.83%


In [45]:
# WIL
wil_score = wil(original_texts, transcription)

In [46]:
print(f"Word Information Lost (WIL): {wil_score * 100:.2f}%")

Word Information Lost (WIL): 71.54%


In [47]:
# Создание датафрейма
info = info.append({
    'MODEL': MODEL_ID,
    'DATASET': test_dataset_cv_12.info.dataset_name,
    'ORIGINAL TEXT': original_texts,
    'PREDICTION TEXT': transcription,
    'WER': wer_score,
    'CER': cer_score,
    'MER': mer_score,
    'WIL': wil_score,
    'SAMPLES': SAMPLES
}, ignore_index=True)
display(info)

/tmp/ipykernel_22161/569941310.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  info = info.append({


,MODEL,DATASET,ORIGINAL TEXT,PREDICTION TEXT,WER,CER,MER,WIL,SAMPLES
0,bond005/wav2vec2-large-ru-golos,common_voice_11_0,"[к сожалению, эти предложения не нашли отражен...",[к сожалению эти предложений не носли отражени...,0.384615,0.071429,0.384615,0.606154,3
1,bond005/wav2vec2-large-ru-golos,common_voice_12_0,"[к сожалению, эти предложения не нашли отражен...",[к сожалению эти предложений не носли отражени...,0.478261,0.110345,0.478261,0.715415,3


## Загрузка датасета `common_voice_14_0`

In [48]:
%%time
test_dataset_cv_14 = load_dataset("mozilla-foundation/common_voice_14_0", LANG_ID, split=f"test[:{SAMPLES}]", trust_remote_code=True)

CPU times: user 295 ms, sys: 11.1 ms, total: 306 ms
Wall time: 3.66 s


In [49]:
if test_dataset_cv_14.features['audio'].sampling_rate != 16_000:
    test_dataset_cv_14 = test_dataset_cv_14.cast_column(
        'audio',
        Audio(sampling_rate=16_000)
    )

### Предсказание

In [50]:
# Освобождаем памяти 50
torch.cuda.empty_cache()

In [51]:
%%time
audio_data = [test_dataset_cv_14[i]['audio']['array'] for i in range(SAMPLES)]

processed = processor(audio_data, sampling_rate=16_000, return_tensors="pt", padding='longest').to(device)

CPU times: user 26.7 ms, sys: 0 ns, total: 26.7 ms
Wall time: 29.9 ms


In [52]:
# Извлекаем логиты напрямую с использованием model.forward
logits = model(processed.input_values, attention_mask=processed.attention_mask).logits

In [53]:
# Оригинальные тексты из датасета
original_texts = [test_dataset_cv_14[i]['sentence'] for i in range(SAMPLES)]

In [54]:
# Транскрибация
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

### Вывод результатов

In [55]:
# Вывод оригинальных и предсказанных текстов
print('-'*100)
for i, (original_text, predicted_text) in enumerate(zip(original_texts, transcription)):
    print(f"Sample {i + 1} \n- Канонический текст: {original_text}")
    print(f"- Предсказанный текст: {predicted_text}")
    print("-" * 100)

----------------------------------------------------------------------------------------------------
Sample 1 
- Канонический текст: К сожалению, эти предложения не нашли отражения в тексте.
- Предсказанный текст: к сожалению эти предложений не носли отражения в тексте
----------------------------------------------------------------------------------------------------
Sample 2 
- Канонический текст: Толпа озвереет, будет тереться, ощетинит ножки стоглавая вошь.
- Предсказанный текст: толпа звереет будет тереться ощути нетножки стоглавая вожь
----------------------------------------------------------------------------------------------------
Sample 3 
- Канонический текст: Если не будет возражений, я буду считать, что Ассамблея согласна с этим предложением.
- Предсказанный текст: если не будет возражений я буду считать что ассамблея согласена с этим предложением
----------------------------------------------------------------------------------------------------


### Тест WER, CER

In [56]:
# Приведем все к нижнему регистру
original_texts = [ref.lower() for ref in original_texts]
transcription = [pred.lower() for pred in transcription]

In [57]:
# Рассчитаем WER
wer_score = wer(original_texts, transcription)

In [58]:
print(f"Word Error Rate (WER): {wer_score * 100:.2f}%")

Word Error Rate (WER): 43.33%


In [59]:
# и CER
cer_score = cer(original_texts, transcription)

In [60]:
print(f"Character Error Rate (CER): {cer_score * 100:.2f}%")

Character Error Rate (CER): 8.82%


In [61]:
# MER
mer_score = mer(original_texts, transcription)

In [62]:
print(f"Match Error Rate (MER): {mer_score * 100:.2f}%")

Match Error Rate (MER): 43.33%


In [63]:
# WIL
wil_score = wil(original_texts, transcription)

In [64]:
print(f"Word Information Lost (WIL): {wil_score * 100:.2f}%")

Word Information Lost (WIL): 67.89%


In [65]:
# Создание датафрейма
info = info.append({
    'MODEL': MODEL_ID,
    'DATASET': test_dataset_cv_14.info.dataset_name,
    'ORIGINAL TEXT': original_texts,
    'PREDICTION TEXT': transcription,
    'WER': wer_score,
    'CER': cer_score,
    'MER': mer_score,
    'WIL': wil_score,
    'SAMPLES': SAMPLES
}, ignore_index=True)
display(info)

/tmp/ipykernel_22161/3254145418.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  info = info.append({


,MODEL,DATASET,ORIGINAL TEXT,PREDICTION TEXT,WER,CER,MER,WIL,SAMPLES
0,bond005/wav2vec2-large-ru-golos,common_voice_11_0,"[к сожалению, эти предложения не нашли отражен...",[к сожалению эти предложений не носли отражени...,0.384615,0.071429,0.384615,0.606154,3
1,bond005/wav2vec2-large-ru-golos,common_voice_12_0,"[к сожалению, эти предложения не нашли отражен...",[к сожалению эти предложений не носли отражени...,0.478261,0.110345,0.478261,0.715415,3
2,bond005/wav2vec2-large-ru-golos,common_voice_14_0,"[к сожалению, эти предложения не нашли отражен...",[к сожалению эти предложений не носли отражени...,0.433333,0.088235,0.433333,0.678889,3


In [66]:
info.to_csv('wav2vec2-large-xlsr-53.csv', index=False)